In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train', 'test', 'train.csv', 'sample_submission.csv']


In [2]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)from keras.models import Sequential
from keras.models import Sequential
from keras.layers import Dropout, Flatten, GlobalAveragePooling2D,Conv2D
from keras.layers import Activation,Dropout,Flatten,Dense,MaxPooling2D,BatchNormalization
from keras.applications import ResNet50
from keras.optimizers import Adam
from keras.models import Model
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator
import math


Using TensorFlow backend.


In [3]:
from keras.models import Sequential
from keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
from keras import applications

In [4]:
X_train = "../input/train/train"
Y_train = "../input/test/test"

In [5]:
len(X_train)

20

In [6]:
train_data = pd.read_csv('../input/train.csv',low_memory=False)
train_data['has_cactus'].value_counts()
train_data.has_cactus = train_data.has_cactus.astype(str)

In [7]:
print("The number of rows in test set is %d"%(len(os.listdir('../input/test/test'))))
print("The number of rows in train set is %d"%(len(os.listdir('../input/train/train'))))

The number of rows in test set is 4000
The number of rows in train set is 17500


In [8]:
#datagen = ImageDataGenerator(rescale=1./255) 
batch_size = 32

In [9]:
train_datagen = ImageDataGenerator(rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,vertical_flip=False,
        fill_mode='nearest')

In [10]:
valid_datagen = ImageDataGenerator(rescale=1/.255)

In [11]:
train_generator=train_datagen.flow_from_dataframe(train_data[:15001],directory=X_train,x_col='id',y_col='has_cactus',
                                           class_mode='binary',batch_size=batch_size,target_size=(224,224))

valid_generator = valid_datagen.flow_from_dataframe(train_data[15000:],directory=X_train,x_col='id',y_col='has_cactus',
                                           class_mode='binary',batch_size=batch_size,target_size=(224,224))

Found 15001 validated image filenames belonging to 2 classes.
Found 2500 validated image filenames belonging to 2 classes.


In [12]:
train_generator.class_indices

{'0': 0, '1': 1}

In [13]:
from keras.applications import ResNet50

In [14]:
base_model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
print('Model Loaded')

base_model.trainable = False


/opt/conda/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 1s 0us/step
Model Loaded


In [15]:
top_model = Sequential()
top_model.add(base_model)
top_model.add(Flatten())
top_model.add(Dense(512))
top_model.add(Activation('relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1))
top_model.add(Activation('sigmoid'))

model=top_model
sgd = optimizers.SGD(lr=1e-3,decay=1e-6,momentum=0.99,nesterov=True)
model.compile(optimizer=sgd,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               51380736  
_________________________________________________________________
activation_50 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 513       
_________________________________________________________________
activation_51 (Activation)   (None, 1)                 0         
Total para

In [16]:
file_path="weights.best.hdf5"

checkpoint = ModelCheckpoint(file_path,monitor='acc', verbose=1, save_best_only=True, mode='max')

early = EarlyStopping(monitor="acc", mode="max", patience=15)

callbacks_list = [checkpoint, early] 

history = model.fit_generator(train_generator,
                    steps_per_epoch= 10,
                    epochs= 100 , verbose=2,
                    validation_data=valid_generator,
                    validation_steps=10,
                    callbacks= callbacks_list)

Epoch 1/100
 - 12s - loss: 0.6337 - acc: 0.8094 - val_loss: 3.2892 - val_acc: 0.7250

Epoch 00001: acc improved from -inf to 0.80937, saving model to weights.best.hdf5
Epoch 2/100
 - 2s - loss: 0.5358 - acc: 0.8969 - val_loss: 2.1135 - val_acc: 0.7875

Epoch 00002: acc improved from 0.80937 to 0.89687, saving model to weights.best.hdf5
Epoch 3/100
 - 5s - loss: 0.3275 - acc: 0.9313 - val_loss: 1.6102 - val_acc: 0.7656

Epoch 00003: acc improved from 0.89687 to 0.93125, saving model to weights.best.hdf5
Epoch 4/100
 - 5s - loss: 0.4311 - acc: 0.9250 - val_loss: 3.3748 - val_acc: 0.7219

Epoch 00004: acc did not improve from 0.93125
Epoch 5/100
 - 5s - loss: 0.3347 - acc: 0.9531 - val_loss: 2.4453 - val_acc: 0.7500

Epoch 00005: acc improved from 0.93125 to 0.95312, saving model to weights.best.hdf5
Epoch 6/100
 - 5s - loss: 0.6227 - acc: 0.9125 - val_loss: 2.9760 - val_acc: 0.7812

Epoch 00006: acc did not improve from 0.95312
Epoch 7/100
 - 5s - loss: 0.5788 - acc: 0.9313 - val_loss: 3